In [1]:

from functools import reduce
import vyper
from openpyxl import load_workbook
from openpyxl.styles import Border, Side, PatternFill, Font, GradientFill, Alignment
from vyper.user import Model
from vyper.utils.tools import StatisticalTools as st
from vyper.user import Model
from vyper.user.explorer import DataProfiler
from vyper.utils.tools import StatisticalTools as st
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from pandas.api.types import is_string_dtype
from openpyxl import Workbook
import pandas as pd
import numpy as np
import re
import statsmodels.api as sm
import math
import warnings
import string

from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.chart import BarChart, Reference, LineChart
from openpyxl.drawing.text import CharacterProperties
from openpyxl.styles import PatternFill, Border, Side, Alignment, Font

In [54]:
## EDITS
def Snapshot_Profile(profile_data, segment_var, segments, segment_names, include, continuous_var_bounds,
                     variable_order, other_segment = False, file = 'Profile', exclude = '',
                     PPT = True, continuous = [], excludeother = False) -> pd.DataFrame:

    profile_data = pd.DataFrame(profile_data)
    if (segments is None):
        segments = pd.unique(profile_data[segment_var])
        other_segment = False
        segment_names = str(segments)
    if (not (isinstance(other_segment, bool))):
        raise ValueError('other must be True or False.')
    if (segment_names is None):
        segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']
    if (len(segments)+other_segment!=len(segment_names)):
        if(len(segments)==len(segment_names)):
            segment_names = [segment_names, 'other']
        else:
            warnings.warn('Incorrect length of names, replacing by segments')
            segment_names = str(segments)
        if (other_segment):
            segment_names = [segment_names, 'other']
    ## output -> [1 2 0]

    varclass = pd.DataFrame()
    for col in profile_data.columns:
        varclass = pd.concat([varclass.reset_index(drop=True), pd.DataFrame([st.classify_variable(profile_data[col])], columns = [col])], axis=1)
#     STATE	ZIP4	ZIP	ID	FIRST_NAME	LAST_NAME	Right_ADDRESS	Right_ZIP	CITY	Right_STATE	...	orderspercustomer_new_2year	spendpercustomers_new_2year	spendperorder_new_1year	orderspercustomer_new_1year	spendpercustomers_new_1year	spendperorder_new_6month	orderspercustomer_new_6month	spendpercustomers_new_6month	y	seg
# 0	categorical	continuous	continuous	continuous	exclude	exclude	exclude  


    # Continuous Variables that were classified as categorical
    varclass[varclass.columns in continuous] = 'continuous'
    if (not (include is None)):
        exclude = varclass.columns[varclass.columns not in include]
    # Variables to exclude
    varclass[varclass.columns in exclude] = 'exclude'
    if (excludeother): 
        varclass[varclass.columns =='Categorical+other'] = 'exclude'
    continuous_var_cuts = dict()

    continuous_var_bounds = [{'MT_BUDGET_MEAL_PLANNERS': [0,50,60,70,75,80,85,90,95,100]}]    
    print(continuous_var_bounds)
    
    for variable in varclass.columns[varclass.iloc[0] == 'continuous']:
        if not continuous_var_bounds:
            continuous_var_bounds = dict()
        if variable in continuous_var_bounds:
            cut_bins = continuous_var_bounds[variable]
            continuous_var_cuts = pd.cut(profile_data[variable], bins=cut_bins)
            # continuous_var_cuts[variable] = continuous_var_bounds[variable]
        else:
            continuous_var_cuts[variable] = st.get_breaks(profile_data.loc[:, variable], nbins=10,
                        squash_extremes=False)
            
    print(continuous_var_cuts)

#     1        (302.4, 454.0]
# 14       (200.0, 250.0]
# 17       (200.0, 250.0]
# 18         (12.0, 50.0]
# 26        (79.0, 100.0]
#               ...      
# 99225     (79.0, 100.0]
# 99231    (100.0, 150.0]
# 99232      (12.0, 50.0]
# 99237      (12.0, 50.0]
# 99253     (0.999, 12.0]
# Name: HOME_EQUITY_LOAN_IN_K, Length: 17564, dtype: category
# Categories (10, interval[float64]): [(0.999, 12.0] < (12.0, 50.0] < (50.0, 79.0] < (79.0, 100.0] ... (200.0, 250.0] < (250.0, 302.4] < (302.4, 454.0] < (454.0, 7500.0]]



    col_order = []
    index_order = []
    ### nEEDS EDITS -> array([2, 0, 1]) Always encode 0 as US
## Change the US population as BASELINE
    for i in range(len(segments)):
        profile_1 = DataProfiler(profile_data[profile_data[segment_var] == segments[i]]).create_profile(number_of_bins = 10,
                                                                            cts_cuts = continuous_var_cuts)
        if (i == 0):
            profile = profile_1
            profile.columns = ["Variable", "Category",  "US_Count","US_Percent"]
            for cols in profile.columns:
                col_order.append(cols)


        else:
            profile_1.columns = ["Variable", "Category",  "Count_"+str(i), "Percent_"+str(i)]
            col_order.append("Count_"+str(i))
            col_order.append("Percent_"+str(i))
            profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')
            profile['US Population vs ' + "Segment_" + str(i)] = (profile["US_Percent"]/profile["Percent_"+str(i)]) * 100
            index_order.append('US Population vs ' + "Segment_" + str(i))

    profile = profile[col_order+index_order]

    for i in profile.columns:

        if ('Count' in i):
            profile[i][profile[i].isna()] = 0
        if ('Percent' in i):
            profile[i][profile[i].isna()] = 0
            
            
    if file:
        import os
        # Make Profile ####
        filesave = file + '.xlsx'
        
        if os.path.exists(filesave):
            os.remove(filesave)
        CreateXLProfile_Snap(profile, savepath = filesave)
        
        if PPT:
            wb = Workbook()

            profile['Category'] = profile['Category'].astype(str)

            all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[0], 0)
            # make new sheet the active sheet we are working on
            all_var_profiling_ws = wb.active

            # remove gridlines from the sheet
            all_var_profiling_ws.sheet_view.showGridLines = False

            # set border line thickness
            thick = Side(border_style="thick", color="000000")
            # medium = Side(border_style="medium", color="000000")
            thin = Side(border_style="thin", color="000000")

            # get max rows and cols of profiling df
            max_rows = profile.shape[0]
            max_cols = profile.shape[1]
            
            input_cols = list(string.ascii_uppercase)

            j = 4
            for z in range(0, len(profile)):
                if (z != 0):
                    if (profile['Variable'].iloc[z] != profile['Variable'].iloc[z - 1]):
                        # create sheet
                        all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[z], 0)
                        # make new sheet the active sheet we are working on
                        all_var_profiling_ws = wb.active

                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = y + 2)
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                        j = 4
                    else:
                        incol = 2
                        for y in range(len(profile.columns)):
                            current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                            incol = incol + 1
                            current_cell.font = Font(bold=True)
                            current_cell.alignment = Alignment(horizontal='center', vertical='center')
                            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                            current_cell.value = profile.columns[y]
                            if y == 0:
                                current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
                            elif y == max_cols - 1:
                                current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                            else:
                                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                            if (y % 2 == 1 and y > 2):
                                current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
                                incol = incol + 1
                                current_cell.font = Font(bold=True)
                                current_cell.alignment = Alignment(horizontal='center', vertical='center')
                                current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
                                current_cell.value = 'Index'
                                if y == max_cols - 1:
                                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
                                else:
                                    current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                                
                incol = 2
                for y in range(len(profile.columns)):
                    current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                    incol = incol + 1
                    current_cell.value = profile.iloc[z, y]
                    if y == 0:
                        current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                    else:
                        current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
                    if (y % 2 == 1 and y > 2):
                        current_cell = all_var_profiling_ws.cell(row = j, column = incol)
                        incol = incol + 1
                        current_cell.value = '=' + input_cols[incol - 3] + str(j) + '/E' + str(j)
                        if y == max_cols - 1:
                            current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                        else:
                            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

                j = j + 1
            
            filesave = file + 'PPT.xlsx'
        
            if os.path.exists(filesave):
                os.remove(filesave)
            
            wb.save(filesave)
            
    return profile

def CreateXLProfile_Snap(profile, savepath):
    wb = Workbook()

    profile['Category'] = profile['Category'].astype(str)

    # create sheet
    all_var_profiling_ws = wb.create_sheet('profile', 0)

    # make new sheet the active sheet we are working on
    all_var_profiling_ws = wb.active

    # remove gridlines from the sheet
    all_var_profiling_ws.sheet_view.showGridLines = False

    # set border line thickness
    thick = Side(border_style="thick", color="000000")
    # medium = Side(border_style="medium", color="000000")
    thin = Side(border_style="thin", color="000000")

    # get max rows and cols of profiling df
    max_rows = profile.shape[0]
    max_cols = profile.shape[1]

    input_rows = range(4, max_rows + 4)
    input_cols = list(string.ascii_uppercase)[1:max_cols + 1]

    y = 0
    for i in input_cols:
        x = "3"
        x = i + x

        all_var_profiling_ws[x].font = Font(bold=True)
        current_cell = all_var_profiling_ws[x]
        current_cell.alignment = Alignment(horizontal='center', vertical='center')
        current_cell.fill = PatternFill("solid", fgColor="A9C4FE")

        if y == 0:
            current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
        elif y == max_cols - 1:
            current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
        else:
            current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

        all_var_profiling_ws[x] = profile.columns[y]
        y = y + 1
    
    print("Worked till here")
    counter = []
    counter_val = 1
    for index,i in enumerate(input_cols[2::]):
        x = "2"
        x = i + x
        if index%2==1:
            all_var_profiling_ws.merge_cells(f"{counter[-1]}:{x}")
            all_var_profiling_ws[counter[-1]].font = Font(bold=True)
            current_cell = all_var_profiling_ws[counter[-1]]
            current_cell.alignment = Alignment(horizontal='center', vertical='center')
            current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
            continue


        if i==input_cols[-1]:
            break
        if i=='D':
            all_var_profiling_ws[x] = "US POPULATION"
            all_var_profiling_ws.merge_cells(start_row=2, start_column=4, end_row=2, end_column=5)
            counter.append(x)
            continue
        all_var_profiling_ws[x] = 'Segment ' + str(counter_val)
        counter_val+=1
        counter.append(x)
        # all_var_profiling_ws.merge_cells(start_row=2, start_column=5, end_row=2, end_column=5)
        # all_var_profiling_ws.merge_cells(f'{x}:D2')

    
    def style_range(ws, cell_range, border=Border(), fill=None, font=None,alignment=None):
        
        """
        Apply styles to a range of cells as if they were a single cell.

        :param ws:  Excel worksheet instance
        :param range: An excel range to style (e.g. A1:F20)
        :param border: An openpyxl Border
        :param fill: An openpyxl PatternFill or GradientFill
        :param font: An openpyxl Font object
        """

        top = Border(top=border.top)
        left = Border(left=border.left)
        right = Border(right=border.right)
        bottom = Border(bottom=border.bottom)

        first_cell = ws[cell_range.split(":")[0]]
        if alignment:
            ws.merge_cells(cell_range)
            first_cell.alignment = alignment

        rows = ws[cell_range]
        if font:
            first_cell.font = font

        for cell in rows[0]:
            cell.border = cell.border + top
        for cell in rows[-1]:
            cell.border = cell.border + bottom

        for row in rows:
            l = row[0]
            r = row[-1]
            l.border = l.border + left
            r.border = r.border + right
            if fill:
                for c in row:
                    c.fill = fill


    border = Border(top=thick, left=thick, right=thick, bottom=thick)

    for range1 in all_var_profiling_ws.merged_cells.ranges:
        style_range(all_var_profiling_ws, str(range1), border=border)

    

    y = 0
    for i in input_cols:
        z = 0
        for j in input_rows:
            all_var_profiling_ws.row_dimensions[j].height = 25
            x = str(j)
            x = i + x
            # print(x)
            current_cell = all_var_profiling_ws[x]
            current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

            if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

            all_var_profiling_ws[x] = profile.iloc[z, y]

            if y == 0:
                current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
            elif y == max_cols - 1:
                current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
                if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
                    current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)


            z = z + 1
        y = y + 1
    

    
    if (savepath != ''):
        wb.save(savepath)

In [18]:
eps_data = pd.read_csv(r"Data/EpsilonResponseModelData_InternalAppend.csv")

In [19]:
eps_data['seg'] = np.random.randint(0, 4, eps_data.shape[0])

In [47]:
continuous_var_data = pd.read_csv('Data/continuous_dictionary.csv')
continuous_var_data

,Variable,Cuts
0,MT_BUDGET_MEAL_PLANNERS,"0,50,60,70,75,80,85,90,95,100"
1,MT_DEBIT_CARD_USER,"0,20,30,40,50,60,70,80,90,100"
2,MT_MEDICAID_POTENTIAL_QUALIFIED_HOUSEHOLD,"0,20,30,40,50,60,70,80,90,100"
3,MT_EVERYDAY_LOW_PRICE_SHOPPERS,"0,50,60,70,80,85,90,92,96,98,100"
4,MT_AMAZON_PRIME_CUSTOMERS,"0,3,5,10,15,20,25,30,40,50,100"
5,MT_ONLINE_SAVINGS_USER,"0,3,6,10,15,20,30,35,45,60,100"
6,MT_WELLNESS_HOUSEHOLDS_HEALTH,"0,4,7,10,15,20,25,30,40,55,100"
7,MT_MEDICARE_ADVANTAGE_PLAN_PURCHASERS,"0,75,100"
8,MT_MAGAZINE_READERS,"0,5,10,15,20,30,40,50,60,75,100"
9,MT_NATIONAL_HEALTHCARE_SUPPORTERS,"0,5,10,15,20,25,30,35,45,55,70,100"


In [48]:
continuous_var_bounds = dict(zip(continuous_var_data.Variable, continuous_var_data.Cuts))
continuous_var_bounds

{'MT_BUDGET_MEAL_PLANNERS': '0,50,60,70,75,80,85,90,95,100',
 'MT_DEBIT_CARD_USER': '0,20,30,40,50,60,70,80,90,100',
 'MT_MEDICAID_POTENTIAL_QUALIFIED_HOUSEHOLD': '0,20,30,40,50,60,70,80,90,100',
 'MT_EVERYDAY_LOW_PRICE_SHOPPERS': '0,50,60,70,80,85,90,92,96,98,100',
 'MT_AMAZON_PRIME_CUSTOMERS': '0,3,5,10,15,20,25,30,40,50,100',
 'MT_ONLINE_SAVINGS_USER': '0,3,6,10,15,20,30,35,45,60,100',
 'MT_WELLNESS_HOUSEHOLDS_HEALTH': '0,4,7,10,15,20,25,30,40,55,100',
 'MT_MEDICARE_ADVANTAGE_PLAN_PURCHASERS': '0,75,100',
 'MT_MAGAZINE_READERS': '0,5,10,15,20,30,40,50,60,75,100',
 'MT_NATIONAL_HEALTHCARE_SUPPORTERS': '0,5,10,15,20,25,30,35,45,55,70,100'}

In [ ]:
continuous_var_bounds = {'MT_BUDGET_MEAL_PLANNERS': [0,50,60,70,75,80,85,90,95,100],
                         'MT_DEBIT_CARD_USER': [0,20,30,40,50,60,70,80,90,100],
                         'MT_MEDICAID_POTENTIAL_QUALIFIED_HOUSEHOLD': [0,20,30,40,50,60,70,80,90,100],
                         'MT_EVERYDAY_LOW_PRICE_SHOPPERS': [0,50,60,70,80,85,90,92,96,98,100],
                         'MT_AMAZON_PRIME_CUSTOMERS': [0,3,5,10,15,20,25,30,40,50,100],
                         'MT_ONLINE_SAVINGS_USER': [0,3,6,10,15,20,30,35,45,60,100],
                         'MT_WELLNESS_HOUSEHOLDS_HEALTH': [0,4,7,10,15,20,25,30,40,55,100],
                         'MT_MEDICARE_ADVANTAGE_PLAN_PURCHASERS': [0,75,100],
                         'MT_MAGAZINE_READERS': [0,5,10,15,20,30,40,50,60,75,100],
                         'MT_NATIONAL_HEALTHCARE_SUPPORTERS': [0,5,10,15,20,25,30,35,45,55,70,100]}

In [55]:
profile = Snapshot_Profile(profile_data = eps_data, segment_var='seg', segments = None, segment_names = None, include = None,
                 continuous_var_bounds = None, variable_order = None, other_segment = False,
                 file = '3-21-test', exclude = [], PPT = True, continuous = [], excludeother = False)

[{'MT_BUDGET_MEAL_PLANNERS': [0, 50, 60, 70, 75, 80, 85, 90, 95, 100]}]
{'ZIP4': 0        (1253.0, 1616.0]
1        (1616.0, 2006.0]
2        (4114.0, 5018.0]
3        (6256.0, 9992.0]
4        (1616.0, 2006.0]
               ...       
99255    (1253.0, 1616.0]
99256    (2401.0, 2836.0]
99257    (5018.0, 6256.0]
99258    (1253.0, 1616.0]
99259    (2006.0, 2401.0]
Name: ZIP4, Length: 99260, dtype: category
Categories (10, interval[float64]): [(0.999, 1253.0] < (1253.0, 1616.0] < (1616.0, 2006.0] < (2006.0, 2401.0] ... (3406.0, 4114.0] < (4114.0, 5018.0] < (5018.0, 6256.0] < (6256.0, 9992.0]], 'ZIP': 0        (11249.0, 11570.0]
1        (11570.0, 19085.0]
2        (6819.999, 7040.0]
3        (10033.0, 10520.0]
4          (7417.0, 7735.0]
                ...        
99255    (11570.0, 19085.0]
99256     (7735.0, 10011.0]
99257    (10520.0, 11102.0]
99258      (7417.0, 7735.0]
99259      (7040.0, 7417.0]
Name: ZIP, Length: 99260, dtype: category
Categories (10, interval[float64]): [(6819.

Worked till here


In [56]:
CreateXLProfile_Snap(profile, '3-21-test.xlsx')

Worked till here


In [ ]:
# def Snapshot_Profile(profile_data, segment_var, segments, segment_names, include, continuous_var_bounds,
#                      variable_order, other_segment = False, file = 'Profile', exclude = '',
#                      PPT = True, continuous = [], excludeother = False) -> pd.DataFrame:
#     profile_data = pd.DataFrame(profile_data)
#     if (segments is None):
#         segments = pd.unique(profile_data[segment_var])
#         other_segment = False
#         segment_names = str(segments)
#     if (not (isinstance(other_segment, bool))):
#         raise ValueError('other must be True or False.')
#     if (segment_names is None):
#         segment_names = str(segments)
#         if (other_segment):
#             segment_names = [segment_names, 'other']
#     if (len(segments)+other_segment!=len(segment_names)):
#         if(len(segments)==len(segment_names)):
#             segment_names = [segment_names, 'other']
#         else:
#             warnings.warn('Incorrect length of names, replacing by segments')
#             segment_names = str(segments)
#         if (other_segment):
#             segment_names = [segment_names, 'other']

#     varclass = pd.DataFrame()
#     for col in profile_data.columns:
#         varclass = pd.concat([varclass.reset_index(drop=True), pd.DataFrame([st.classify_variable(profile_data[col])], columns = [col])], axis=1)

#     # Continuous Variables that were classified as categorical
#     varclass[varclass.columns in continuous] = 'continuous'
#     if (not (include is None)):
#         exclude = varclass.columns[varclass.columns not in include]
#     # Variables to exclude
#     varclass[varclass.columns in exclude] = 'exclude'
#     if (excludeother): varclass[varclass.columns =='Categorical+other'] = 'exclude'


#     continuous_var_cuts = dict()
#     for variable in varclass.columns[varclass.iloc[0] == 'continuous']:
#         if not continuous_var_bounds:
#             continuous_var_bounds = dict()
#         if variable in continuous_var_bounds:
#             continuous_var_cuts[variable] = continuous_var_bounds[variable]
#         else:
#             continuous_var_cuts[variable] = st.get_breaks(profile_data.loc[:, variable], nbins=10,
#                      squash_extremes=False)

#     for i in range(len(segments)):
#         profile_1 = DataProfiler(profile_data[profile_data[segment_var] == segments[i]]).create_profile(number_of_bins = 10,
#                                                                           cts_cuts = continuous_var_cuts)
#         if (i == 0):
#             profile = profile_1
#         else:
#             profile = pd.merge(profile, profile_1, on = ['Variable', 'Category'], how = 'outer')

#     for i in profile.columns:
#         print(i)
#         print(profile[i])
#         if ('Count' in i):
#             profile[i][profile[i].isna()] = 0
#         if ('Percent' in i):
#             profile[i][profile[i].isna()] = 0
            
            
#     if file:
#         import os
#         # Make Profile ####
#         filesave = file + '.xlsx'
        
#         if os.path.exists(filesave):
#             os.remove(filesave)
#         CreateXLProfile_Snap(profile, savepath = filesave)
        
#         if PPT:
#             wb = Workbook()

#             profile['Category'] = profile['Category'].astype(str)

#             all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[0], 0)
#             # make new sheet the active sheet we are working on
#             all_var_profiling_ws = wb.active

#             # remove gridlines from the sheet
#             all_var_profiling_ws.sheet_view.showGridLines = False

#             # set border line thickness
#             thick = Side(border_style="thick", color="000000")
#             # medium = Side(border_style="medium", color="000000")
#             thin = Side(border_style="thin", color="000000")

#             # get max rows and cols of profiling df
#             max_rows = profile.shape[0]
#             max_cols = profile.shape[1]
            
#             input_cols = list(string.ascii_uppercase)

#             j = 4
#             for z in range(0, len(profile)):
#                 if (z != 0):
#                     if (profile['Variable'].iloc[z] != profile['Variable'].iloc[z - 1]):
#                         # create sheet
#                         all_var_profiling_ws = wb.create_sheet(profile['Variable'].iloc[z], 0)
#                         # make new sheet the active sheet we are working on
#                         all_var_profiling_ws = wb.active

#                         for y in range(len(profile.columns)):
#                             current_cell = all_var_profiling_ws.cell(row = 3, column = y + 2)
#                             current_cell.font = Font(bold=True)
#                             current_cell.alignment = Alignment(horizontal='center', vertical='center')
#                             current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
#                             current_cell.value = profile.columns[y]
#                             if y == 0:
#                                 current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#                             elif y == max_cols - 1:
#                                 current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#                             else:
#                                 current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
#                         j = 4
#                     else:
#                         incol = 2
#                         for y in range(len(profile.columns)):
#                             current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
#                             incol = incol + 1
#                             current_cell.font = Font(bold=True)
#                             current_cell.alignment = Alignment(horizontal='center', vertical='center')
#                             current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
#                             current_cell.value = profile.columns[y]
#                             if y == 0:
#                                 current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#                             elif y == max_cols - 1:
#                                 current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#                             else:
#                                 current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
#                             if (y % 2 == 1 and y > 2):
#                                 current_cell = all_var_profiling_ws.cell(row = 3, column = incol)
#                                 incol = incol + 1
#                                 current_cell.font = Font(bold=True)
#                                 current_cell.alignment = Alignment(horizontal='center', vertical='center')
#                                 current_cell.fill = PatternFill("solid", fgColor="A9C4FE")
#                                 current_cell.value = 'Index'
#                                 if y == max_cols - 1:
#                                     current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#                                 else:
#                                     current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)
                                
#                 incol = 2
#                 for y in range(len(profile.columns)):
#                     current_cell = all_var_profiling_ws.cell(row = j, column = incol)
#                     incol = incol + 1
#                     current_cell.value = profile.iloc[z, y]
#                     if y == 0:
#                         current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
#                     else:
#                         current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
#                     if (y % 2 == 1 and y > 2):
#                         current_cell = all_var_profiling_ws.cell(row = j, column = incol)
#                         incol = incol + 1
#                         current_cell.value = '=' + input_cols[incol - 3] + str(j) + '/E' + str(j)
#                         if y == max_cols - 1:
#                             current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
#                         else:
#                             current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

#                 j = j + 1
            
#             filesave = file + 'PPT.xlsx'
        
#             if os.path.exists(filesave):
#                 os.remove(filesave)
            
#             wb.save(filesave)
            
#     return profile

In [ ]:
# def CreateXLProfile_Snap(profile, savepath):
#     wb = Workbook()

#     profile['Category'] = profile['Category'].astype(str)

#     # create sheet
#     all_var_profiling_ws = wb.create_sheet('profile', 0)

#     # make new sheet the active sheet we are working on
#     all_var_profiling_ws = wb.active

#     # remove gridlines from the sheet
#     all_var_profiling_ws.sheet_view.showGridLines = False

#     # set border line thickness
#     thick = Side(border_style="thick", color="000000")
#     # medium = Side(border_style="medium", color="000000")
#     thin = Side(border_style="thin", color="000000")

#     # get max rows and cols of profiling df
#     max_rows = profile.shape[0]
#     max_cols = profile.shape[1]

#     input_rows = range(4, max_rows + 4)
#     input_cols = list(string.ascii_uppercase)[1:max_cols + 1]

#     y = 0
#     for i in input_cols:
#         x = "3"
#         x = i + x

#         all_var_profiling_ws[x].font = Font(bold=True)
#         current_cell = all_var_profiling_ws[x]
#         current_cell.alignment = Alignment(horizontal='center', vertical='center')
#         current_cell.fill = PatternFill("solid", fgColor="A9C4FE")

#         if y == 0:
#             current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#         elif y == max_cols - 1:
#             current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)
#         else:
#             current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

#         all_var_profiling_ws[x] = profile.columns[y]
#         y = y + 1

#     y = 0
#     for i in input_cols:
#         z = 0
#         for j in input_rows:
#             all_var_profiling_ws.row_dimensions[j].height = 25
#             x = str(j)
#             x = i + x
#             # print(x)
#             current_cell = all_var_profiling_ws[x]
#             current_cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)

#             if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
#                 current_cell.border = Border(top=thick, left=thin, right=thin, bottom=thin)

#             all_var_profiling_ws[x] = profile.iloc[z, y]

#             if y == 0:
#                 current_cell.border = Border(top=thin, left=thick, right=thin, bottom=thin)
#                 if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
#                     current_cell.border = Border(top=thick, left=thick, right=thin, bottom=thin)
#             elif y == max_cols - 1:
#                 current_cell.border = Border(top=thin, left=thin, right=thick, bottom=thin)
#                 if profile['Variable'].iloc[z] != profile['Variable'].iloc[z-1]:
#                     current_cell.border = Border(top=thick, left=thin, right=thick, bottom=thin)


#             z = z + 1
#         y = y + 1
    

    
#     if (savepath != ''):
#         wb.save(savepath)